In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("/kaggle/input/level1-stack/level1_train_fold.csv")
df_test = pd.read_csv("/kaggle/input/level1-stack/level1_test.csv")
submission = pd.read_csv("/kaggle/input/30-days-of-ml/sample_submission.csv")

In [3]:
useful_features = ['pred_1', 'pred_2', 'pred_3']
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    params = {
            'n_estimators': 10000,
            'lambda': 0.002737255187493384,
             'alpha': 7.1401361029365435e-06,
             'colsample_bytree': 0.0943116642365347,
             'subsample': 0.14136126196670723,
             'learning_rate': 0.013736831072935482,
             'max_depth': 1,
             'min_child_weight': 114
            }
    
    # Define the model 
    model = XGBRegressor(random_state=fold, n_jobs=4, **params)

    # Train the model (will take about 10 minutes to run)
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

[0]	validation_0-rmse:7.67397
[1000]	validation_0-rmse:0.71545
[1128]	validation_0-rmse:0.71545
0 0.7154446335698168
[0]	validation_0-rmse:7.67067
[929]	validation_0-rmse:0.71567
1 0.7156642571897134
[0]	validation_0-rmse:7.66850
[862]	validation_0-rmse:0.71734
2 0.7173161748118784
[0]	validation_0-rmse:7.67068
[946]	validation_0-rmse:0.71744
3 0.7174361980275737
[0]	validation_0-rmse:7.67614
[1000]	validation_0-rmse:0.71542
[1137]	validation_0-rmse:0.71541
4 0.7153932978374538
0.7162509122872871 0.0009240604580592287


In [4]:
submission['target'] = np.mean(np.column_stack(final_predictions), axis=1)
submission.to_csv('submission.csv', index=False)